---
# 남북한 기사 워드클라우드
작성자: 김지민, 김영성

이 노트북은 남한(국방일보) 및 북한(KCNA) 기사 데이터를 기반으로  
워드클라우드를 생성하고 시각화하는 전체 과정을 수행합니다.

## 주요 내용
- 불용어(stopwords) 설정 및 필터링
- 형태소 분석기(Okt)를 사용해 명사, 동사, 형용사 추출
- 단어 빈도에 따라 색상 밝기를 조절하는 함수 정의 (`get_color_func`)
- 기사 파일명 및 소속(남/북한)에 따라 자동으로 색상 조건 부여  
  - 남한: 1~14번 초록 / 15번 파랑  
  - 북한: 1~14번 빨강 / 15번 파랑
- 워드클라우드 이미지를 PNG로 저장 (자동 경로 및 이름 설정)
- 전체 기사 파일(`data/남한기사/`, `data/북한기사/`)을 불러와 일괄 처리 가능

## 저장 위치
- 남한 기사 워드클라우드: `data/워드클라우드_남한/`
- 북한 기사 워드클라우드: `data/워드클라우드_북한/`
- 저장 형식 예시: `1_남한기사_러시아_wordcloud.png`
---

In [1]:
import pandas as pd
import glob
import os
import platform
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from konlpy.tag import Okt

In [2]:
if platform.system() == 'Windows':
    font_path = 'C:/Windows/Fonts/malgun.ttf'
elif platform.system() == 'Darwin':
    font_path = '/System/Library/Fonts/AppleGothic.ttf'
else:
    print('⚠️ Unknown system')

-------

## 1. 불용어 지정

In [3]:
stopwords = {
    '하였다', '하여', '하는', '하고있다', '다음', '였다', '되였다', '하기', '있다', '많은',
    '할수', '하면서', '대해', '한다', '하시였다', '따라', '하며', '누구', '때문', '위해',
    '어떤', '무슨', '위대한', '있는', '지금', '아니라', '그것', '입니다', '모든', '없다',
    '하신', '대하', '되어', '되여', '아니라', '바로', '지난해', '하면', '자기', '사실', '지난', 
    '즈음', '되는', '의해', '되고', '않는', '합니다', '한다고', '같은', '있었다', '여러', '해야',
    '없는', '더욱', '대한', '이어', '가장', '관련', '오늘', '이런', '하게', '비롯',
    '다른', '높이', '하고', '계속', '이니', '하고있는', '자주', '우리', '여기', '최근', '이번', '있습니다',
    '전체', '되었습니다', '했습니다', '또한', '깊은','되고있다', '로부터', '하지', '올해', '보고', '어떻게',
    '된다', '이러한', '있게', '그이', '않고', '하였습니다', '그대로', '했다', '무단', '배포', '금지', '저작권',
    '공유', '됐다', '통해', '기사', '댓글', '스크랩', '국방일보', '이었다', '밝혔다'
}

----

## 2. 단어 빈도 기반 색상 지정 함수 (`get_color_func`)
- 단어의 빈도 비율에 따라 색상 밝기를 조절하는 함수
- 기준 색상(`base_rgb`)을 전달받아, 자주 등장한 단어일수록 진한 색으로 표현
- 워드클라우드 시각화 시 `color_func`로 전달되어 단어별 색상 지정

In [4]:
def get_color_func(word_freq_dict, base_rgb):
    max_freq = max(word_freq_dict.values())

    def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
        freq = word_freq_dict.get(word, 0)        # word가 딕셔너리에 있으면 그 값 반환, 없으면 0 반환
        ratio = freq / max_freq                   # 해당 단어의 빈도가 최대 빈도의 몇 %인지 (0~1 사이의 값)
        
        r = int(255 - (255 - base_rgb[0]) * ratio) # ratio가 높을수록 base_rgb에 가까운 색상 생성
        g = int(255 - (255 - base_rgb[1]) * ratio)
        b = int(255 - (255 - base_rgb[2]) * ratio)
        return (r, g, b)

    return color_func

-----

## 3. 워드클라우드 생성 함수 (`draw_wordcloud_from_file`)

- 지정된 CSV 파일을 읽어 특정 열(`Content`)에서 텍스트 추출
- Okt 형태소 분석기로 명사, 동사, 형용사만 추출
- 불용어를 제거하고 단어 빈도를 계산하여 워드클라우드 생성
- 파일명 기반으로 기사 번호와 기사 종류(남한/북한)를 구분 (`파일명: 1_북한기사_전체.csv, 1_남한기사_러시아.csv`)
- 기사 번호와 소속에 따라 워드클라우드 색상 자동 분기
  - 남한 1~14번: 초록
  - 남한 15번: 파랑
  - 북한 1~14번: 빨강
  - 북한 15번: 파랑
- 결과 이미지를 `data/워드클라우드_남한` 또는 `data/워드클라우드_북한` 폴더에 자동 저장

In [5]:
def draw_wordcloud_from_file(file_path):
    df = pd.read_csv(file_path)
    text_list = (
        df['Title'].fillna('') + ' ' + df['Content'].fillna('') # NaN을 빈문자열로 바꿔줌(NaN이 하나라도 껴있으면 NaN으로 텍스트 누락됨)
    ).astype(str).tolist()

    # 형태소 분석 및 단어 추출
    okt = Okt()
    tokens = []
    for line in text_list:
        token_pos = okt.pos(line)
        tokens += [token for token, tag in token_pos if tag in ['Noun', 'Verb', 'Adjective']]

    # 파일명 기반으로 번호와 기사 종류 추출
    filename = os.path.basename(file_path)
    file_number = int(filename.split('_')[0])
    is_south = '남한기사' in filename
    nation = filename.split('_')[2].split('.')[0]
    
    # 동적으로 제거할 단어 결정
    if '전체' in filename:
        dynamic_stopwords = stopwords
    elif nation == '한국':
        dynamic_stopwords = stopwords.union({'한국', '남조선'})
    elif nation == '러시아':
        dynamic_stopwords = stopwords.union({'러시아', '로씨야'})
    else:
        dynamic_stopwords = stopwords.union({nation})

    # 불용어 적용
    tokens = [token for token in tokens if len(token) >= 2 and token not in dynamic_stopwords]
    cnt = Counter(tokens)

    # 색상 조건 설정
    if is_south:
        base_rgb = (0, 100, 0) if file_number <= 14 else (0, 0, 139)
    else:
        base_rgb = (139, 0, 0) if file_number <= 14 else (0, 0, 139)

    color_func = get_color_func(cnt, base_rgb=base_rgb) # 색상을 계산해주는 함수 반환 (wordcloud가 단어 하나씩 칠할 때 호출하는 콜백 함수)

    # 저장 디렉토리 및 파일명 설정
    save_dir = 'data/워드클라우드_남한' if is_south else 'data/워드클라우드_북한'
    save_name = filename.split('.')[0]
    save_path = f'{save_dir}/{save_name}_wordcloud.png'
    os.makedirs(save_dir, exist_ok=True) # 폴더 없으면 생성

    # 워드클라우드 생성 및 저장
    wc = WordCloud(
        font_path=font_path,
        max_font_size=100,
        width=800,
        height=400,
        background_color='white',
        color_func=color_func #워드클라우드 각 단어에 대해 color_func()호출해서 색상 계산
    ).generate_from_frequencies(cnt)

    plt.figure(figsize=(10, 5))
    plt.axis('off')
    plt.imshow(wc, interpolation='bilinear')
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"저장 완료: {save_path}")

------

## 4. 전체 워드클라우드 일괄 처리

- `data/남한기사/`, `data/북한기사/` 경로에 있는 모든 CSV 파일을 정렬
- 각 파일에 대해 `draw_wordcloud_from_file()` 함수를 호출하여 순차적으로 워드클라우드 생성

In [7]:
# 남한 기사 처리
south_path = 'data/남한기사/'
south_files = sorted(glob.glob(f'{south_path}*_남한기사_*.csv'))

for file in south_files:
    print(f'Processing South: {file}')
    draw_wordcloud_from_file(file)

# 북한 기사 처리
north_path = 'data/북한기사/'
north_files = sorted(glob.glob(f'{north_path}*_북한기사_*.csv'))

for file in north_files:
    print(f'Processing North: {file}')
    draw_wordcloud_from_file(file)

Processing South: data/남한기사/10_남한기사_러시아.csv


저장 완료: data/워드클라우드_남한/10_남한기사_러시아_wordcloud.png
Processing South: data/남한기사/10_남한기사_미국.csv
저장 완료: data/워드클라우드_남한/10_남한기사_미국_wordcloud.png
Processing South: data/남한기사/10_남한기사_북한.csv
저장 완료: data/워드클라우드_남한/10_남한기사_북한_wordcloud.png
Processing South: data/남한기사/10_남한기사_전체.csv
저장 완료: data/워드클라우드_남한/10_남한기사_전체_wordcloud.png
Processing South: data/남한기사/10_남한기사_중국.csv
저장 완료: data/워드클라우드_남한/10_남한기사_중국_wordcloud.png
Processing South: data/남한기사/11_남한기사_러시아.csv
저장 완료: data/워드클라우드_남한/11_남한기사_러시아_wordcloud.png
Processing South: data/남한기사/11_남한기사_미국.csv
저장 완료: data/워드클라우드_남한/11_남한기사_미국_wordcloud.png
Processing South: data/남한기사/11_남한기사_북한.csv
저장 완료: data/워드클라우드_남한/11_남한기사_북한_wordcloud.png
Processing South: data/남한기사/11_남한기사_전체.csv
저장 완료: data/워드클라우드_남한/11_남한기사_전체_wordcloud.png
Processing South: data/남한기사/11_남한기사_중국.csv
저장 완료: data/워드클라우드_남한/11_남한기사_중국_wordcloud.png
Processing South: data/남한기사/12_남한기사_러시아.csv
저장 완료: data/워드클라우드_남한/12_남한기사_러시아_wordcloud.png
Processing South: data/남한기사/12_남한기사_미국.csv
저장 완료